In [1]:
import pymc as pm
import numpy as np
import arviz as az
import pandas as pd

%load_ext lab_black
%load_ext watermark

# Revisiting UK coal mining disasters

This example demonstrates ...

Adapted from [unit 10: disasters.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit10/disasters.odc).

Data can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/r.txt).

## Associated lecture video: Unit 10 Lesson 3

In [2]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed?v=xomK4tcePmc&list=PLv0FeK5oXK4l-RdT6DWJj0_upJOG2WKNO&index=100" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

## Problem statement

Change Point Analysis, discussed in Unit 5 (Gibbs Sampler).

British coal mine disaster data by year (1851-1962)  

The 112 data points represent the numbers of coal-mining disasters involving 10 or more men killed per year between  1851 and 1962. 
 
Based on the observation that the there was a significant decrease around 1900, it is suitable to apply a change-point model to divide the whole dataset into two periods; each period with its own distribution of number of disasters.
 
The data set was compiled by Maguire, Pearson and Wynn in 1952 and 
updated by Jarrett (1978). This data have been used by a number of authors to illustrate various techniques that can be applied to point processes


 Maguire, B. A., Pearson, E. S. and Wynn, A. H. A. (1952). The time intervals between industrial accidents.   Biometrika, 39, 168†180.

 Jarrett, R.G. (1979). A note on the intervals between coal-mining disasters. Biometrika, 66, 191-193. 

 Carlin, Gelfand, and Smith (1992) Heirarchical Bayesian Analysis of Changepoint Problems. Applied Statistics, 41, 389-405.
